In [5]:
import os
import cv2
import numpy as np
import imageio
from PIL import Image

In [6]:
def preprocess_frame(frame):
    
    # Apply filtering, smoothing, sharpening, and adjusting brightness and contrast
    blurred = cv2.GaussianBlur(frame, (5, 5), 0)
    kernel = np.array([[0, -1, 0], [-1, 5, -1], [0, -1, 0]])
    sharpened = cv2.filter2D(blurred, -1, kernel)
    
    alpha = 1.1  # Contrast control
    beta = 15    # Brightness control
    adjusted = cv2.convertScaleAbs(sharpened, alpha=alpha, beta=beta)
    
    # Normalize the image
    normalized = cv2.normalize(adjusted, None, 0, 255, cv2.NORM_MINMAX)
    
    return normalized

In [7]:
def create_gif(frames, gif_path):
    with imageio.get_writer(gif_path, mode='I', duration=0.1) as writer:
        for frame in frames:
            rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            writer.append_data(rgb_frame)

def display_gif(gif_path):
    return Image(filename=gif_path)

def save_frames_as_avi(frames, output_path, fps=30):
    if not frames:
        print("No frames to save.")
        return
    
    height, width, layers = frames[0].shape
    size = (width, height)
    
    out = cv2.VideoWriter(output_path, cv2.VideoWriter_fourcc(*'XVID'), fps, size)
    
    for frame in frames:
        out.write(frame)
    
    out.release()
    print(f"Video saved as {output_path}")

In [22]:
def ucf50_preprocessing(data_dir, output_dir, class_names, number_of_frames):
    sift = cv2.SIFT_create()
    feature_matcher = cv2.BFMatcher()
    for class_name in class_names:
        class_input_path = os.path.join(data_dir, class_name)
        output_class_path = os.path.join(output_dir, class_name)
        os.makedirs(output_class_path, exist_ok=True)
        
        for video_name in os.listdir(class_input_path):
            video_path = os.path.join(class_input_path, video_name)
            cap = cv2.VideoCapture(video_path)
            relevant_frames = []
            all_frames = []
            prev_descriptors = None
            total_frames = 0
            
            while True:
                ret, frame = cap.read()
                if not ret:
                    break

                processed_frame = preprocess_frame(frame)
                keypoints, descriptors = sift.detectAndCompute(processed_frame, None)
                
                if descriptors is not None and len(descriptors) > 0:
                    if prev_descriptors is None:
                        relevant_frames.append(frame)
                    else:
                        matches = feature_matcher.knnMatch(prev_descriptors, descriptors, k=2)
                        good_matches = []
                        for match in matches:
                            if len(match) == 2:  # Ensure at least two matches are returned
                                m, n = match
                                if m.distance < 0.75 * n.distance:
                                    good_matches.append(m)
                        total_frames += 1
                        all_frames.append((total_frames, len(good_matches), frame))
                    
                    prev_descriptors = descriptors
            
            cap.release()

            all_frames.sort(key=lambda x: x[1])
            selected_frames = all_frames[:min(len(all_frames), 2 * number_of_frames)]
            selected_frames.sort(key=lambda x: x[0])
            relevant_frames = [frame[2] for frame in selected_frames]

            print()
            # Ensure minimum number of frames is at least number_of_frames
            if len(relevant_frames) < number_of_frames:
                print(f"Selected frames: {len(relevant_frames)}")
                needed_frames = number_of_frames - len(relevant_frames)
                new_frames = [frame[2] for frame in all_frames[:needed_frames]]
                relevant_frames.extend(new_frames)
            
            print(f"Video: {video_name}")
            print(f"Total frames: {total_frames}")
            print(f"Selected frames: {len(relevant_frames)}\n")
            
            # Save the selected frames as an AVI video
            output_video_path = os.path.join(output_class_path, video_name)
            save_frames_as_avi(relevant_frames, output_video_path)

In [13]:
data_dir = 'UCF50'
output_dir = 'SIFT/'


In [23]:
class_names = os.listdir(data_dir)

In [24]:
ucf50_preprocessing(data_dir, output_dir, class_names, number_of_frames=30)


Video: v_Skijet_g25_c02.avi
Total frames: 249
Selected frames: 60

Video saved as SIFT/Skijet/v_Skijet_g25_c02.avi

Video: v_Skijet_g13_c02.avi
Total frames: 209
Selected frames: 60

Video saved as SIFT/Skijet/v_Skijet_g13_c02.avi

Video: v_Skijet_g03_c02.avi
Total frames: 315
Selected frames: 60

Video saved as SIFT/Skijet/v_Skijet_g03_c02.avi

Video: v_Skijet_g03_c03.avi
Total frames: 315
Selected frames: 60

Video saved as SIFT/Skijet/v_Skijet_g03_c03.avi

Video: v_Skijet_g13_c03.avi
Total frames: 209
Selected frames: 60

Video saved as SIFT/Skijet/v_Skijet_g13_c03.avi

Video: v_Skijet_g25_c01.avi
Total frames: 306
Selected frames: 60

Video saved as SIFT/Skijet/v_Skijet_g25_c01.avi

Video: v_Skijet_g24_c04.avi
Total frames: 209
Selected frames: 60

Video saved as SIFT/Skijet/v_Skijet_g24_c04.avi

Video: v_Skijet_g16_c03.avi
Total frames: 209
Selected frames: 60

Video saved as SIFT/Skijet/v_Skijet_g16_c03.avi

Video: v_Skijet_g06_c03.avi
Total frames: 250
Selected frames: 60

Vide